In [1]:
%pip install --upgrade pip
%pip install boto3 --force-reinstall --quiet
%pip install botocore --force-reinstall --quiet
%pip install langchain --force-reinstall --quiet

  Using cached pip-24.0-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.0-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.2 requires botocore<1.34.52,>=1.34.41, but you have botocore 1.34.112 which is incompatible.
opensearch-py 2.5.0 requires urllib3<2,>=1.26.18, but you have urllib3 2.2.1 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sourc

In [1]:
store -r kb_id

In [2]:
import boto3
import pprint
from botocore.client import Config
import json

pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = session.region_name
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config, region_name = region)
print(region)

us-east-1


In [3]:
def retrieve(query, kbId, numberOfResults=10):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults
            }
        }
    )

In [4]:
def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults:
        contexts.append(retrievedResult['content']['text'])
    return contexts

In [5]:
def findProduct(product, numberOfResults=10):
    response = retrieve(product, kb_id, numberOfResults)
    contexts = get_contexts(response['retrievalResults'])
    pp.pprint(contexts)
    print()

    prompt = f"""Human: You are an assistant at a hardware store helping a customer find an  item.  The list of items available at the store is 
    enclosed in <context> tags. The item that the customer is looking for is enclosed in <query> tags. The customer's description of the item might 
    not exactly match the description in your list. Make sure that you give the customer an item of the same size/caliber/measurement that they are 
    looking for. If you find an item that is the exact item that the customer is looking for, respond only with the item's description from your list.
    If you do not find the exact item, respond with a list of similar items.

    <context>
    {contexts}
    </context>

    <query>
    {product}
    </query>

    Assistant:
    """

    body = json.dumps({"prompt": prompt, "max_tokens_to_sample": 100})
    modelId = "anthropic.claude-instant-v1"
    accept = "application/json"
    contentType = "application/json"

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    responsebody = json.loads(response.get("body").read())
    return responsebody.get("completion")

In [6]:
def findListOfProducts(lst, numberOfResults=50):
    query = ""
    for i in lst:
        query += "Item: " + i + "\n"

    response = retrieve(query, kb_id, numberOfResults)
    contexts = get_contexts(response['retrievalResults'])
    pp.pprint(contexts)
    print()

    prompt = f"""Human: You are an assistant at a hardware store helping a customer find items. You have a list of the items available at the 
    hardware store. The items' descriptions are in spanish. The list of items is enclosed in <context> tags. The items that the customer is looking
    for are enclosed in <query> tags. The customer's description of the items might not exactly match the descriptions in your list. Make sure that
    you give the customer items of the same size/caliber/measurement that they are looking for. Respond with a list of the customer's description of
    each item. For each item in this list, include your suggestions for that item. If you do not find a certain item, recommend other similar items. 
    If there are no similar items that can be recommended, include in your response a list of the items that were not found.

    <context>
    {contexts}
    </context>

    <query>
    {query}
    </query>

    Assistant:
    """

    body = json.dumps({"prompt": prompt, "max_tokens_to_sample": 600})
    modelId = "anthropic.claude-instant-v1"
    accept = "application/json"
    contentType = "application/json"

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    responsebody = json.loads(response.get("body").read())
    return responsebody.get("completion")

In [10]:
print(findProduct("Pala redonda c/mgo y pryp4truper", 60))

[ 'Jgo. De 9 llaves allen mm extralargas',
  'Pala azada, redonda, # 5',
  'Pala azada, redonda, # 6',
  "Perico profesional, pavonado 4', Truper Expert en tarjeta",
  'Guantes uso gral. Carnaza y loneta, doble refuerzo, chicos',
  "Perico profesional, pavonado 10', Truper Expert en tarjeta",
  'Guantes de nylon recubiertos con poliuretano, chicos',
  "Perico profesional, pavonado 24', Truper Expert en tarjeta",
  'Repuesto para podadora POMA-15, cuna',
  "MANGUERA 4 CAPAS  DE 5/8' REFORZADA",
  "MANGUERA 4 CAPAS  DE 5/8' REFORZADA",
  "Perico profesional, pavonado 15', Truper Expert en tarjeta",
  "Perico profesional, pavonado 6', Truper Expert en tarjeta",
  "Perico profesional, pavonado 12', Truper Expert en tarjeta",
  'Juego 11 brocasierras bimetalicas, Truper Expert',
  "Nivel profesional Truper Expert 24'",
  'Carretilla 6 ft3, llanta imponchable',
  "Nivel profesional Truper Expert 48'",
  "Perico profesional, pavonado 8', Truper Expert en tarjeta",
  "Juego dados con sujecion,

In [18]:
print(findProduct("Pala redonda c/ mango y pryp4truper"))

[ "Pala redonda, tipo Colombia, # 4, mango 'Y'",
  'Pala redonda, tipo Colombia, # 4, mango largo',
  "Lima plana, bastarda, 6' con mango doble inyeccion",
  "Lima plana, bastarda, 8' con mango doble inyeccion",
  "Pala T-1000 redonda puno 'Y' mango 18",
  "Lima redonda, bastarda, 8', con mango doble inyeccion",
  "Lima redonda, bastarda, 6', con mango doble inyeccion",
  "Lima plana bastarda 10' con mango doble inyeccion",
  "Lima triangulo pesado 4' con mango",
  "Llana, mango plastico, canto dentado cuadrado,6 remaches,11'"]

 "Pala redonda, tipo Colombia, # 4, mango 'Y'"


In [11]:
print(findProduct("Broca p/concreto 1/4x12\" BCT1/4x12 Truper"))

[ "Broca para concreto, 5/16 x 12'",
  "Broca para concreto, 3/8 x 12'",
  "Broca para concreto, 3/16 x 4'",
  "Broca para concreto, 1/4 x 12'",
  "Broca para concreto, 5/8 x 12'",
  "Broca para concreto, 1/2 x 12'",
  "Broca para concreto, 7/8 x 12'",
  "Broca para concreto, 5/16 x 4'",
  "Broca para concreto, 3/4 x 12'",
  "Broca para concreto, 3/16 x 6'"]

 Broca para concreto, 1/4 x 12'"


In [13]:
print(findProduct("Bolsa de 50 remaches 4.8 x 14.5 mm", 30))

[ 'Remaches, bolsa con 50 piezas, 3.2x8.0mm',
  'Remaches, bolsa con 50 piezas, 4x7.10mm',
  'Remaches, bolsa con 50 piezas, 3.2x6.4mm',
  'Remaches, bolsa con 50 piezas, 3.2x9.6mm',
  'Remaches, bolsa con 50 piezas, 4x10.20mm',
  'Remaches, bolsa con 50 piezas,4.8x24.1mm',
  'Remaches, bolsa con 50 piezas, 4x16.50mm',
  'Remaches, bolsa con 50 piezas,4.8x8.0mm',
  'Remaches, bolsa con 50 piezas,6.4x19.1mm',
  'Remaches, bolsa con 50 piezas, 4x13.4mm',
  'Remaches, bolsa con 50 piezas,4.8x11.0mm',
  'Remaches, bolsa con 50 piezas,3.2x12.7mm',
  'Remaches, bolsa con 50 piezas,4.8x21mm',
  'Remaches, bolsa con 50 piezas,4.8x14.5mm',
  'Remaches, bolsa con 50 piezas,4.8x14.5mm',
  'Remaches, bolsa con 50 piezas,4.8x14.5mm',
  'Remaches, bolsa con 50 piezas,6.4x12.7mm',
  'Remaches, bolsa con 50 piezas,6.4x15.9mm',
  'Remaches, bolsa con 50 piezas,4.8x17.7mm',
  'Remaches,bolsa con 50 piezas,3.2x11.10mm',
  'Remaches,bolsa con 50 piezas,3.2x16.1mm',
  "Barreta de punta de 1-1/4' x 175 cm",

In [14]:
print(findProduct('Broca p/router pecho paloma 1-1/8"x13mm BROUPP11/8X Truper'))

[ "Broca para router, pecho paloma, 1-3/8' con balero",
  "Broca para router, pecho paloma, 1-1/8' con balero",
  "Broca para router, recta 2 filos, 1/4', mediana",
  "Broca para router, recta 2 filos, 1/4', larga",
  "Broca para router, bocel cuarto, 1-1/8' con balero",
  "Broca para router, clasica, 1-3/8' con balero",
  "Broca para router, collarin, 7/8' con balero",
  "Broca para router, corte recto, 1/2' con balero",
  "Broca para router, doble romana, 1-3/8' con balero",
  "Broca para router, redondeado, 1-1/4' con balero"]

 "Broca para router, pecho paloma, 1-1/8' con balero"


In [17]:
print(findProduct('Disco diamante 7x7/8 rin continuo uso gral ddt-804c truper'))

[ "Disco de diamante, rin continuo, 9'",
  "Disco de diamante, rin continuo, 4'",
  "Disco de diamante, rin continuo, 7'",
  "Disco de diamante, rin continuo, 10'",
  "Disco de diamante, rin segmentado, 9'",
  "Disco de diamante, rin segmentado, 7'",
  "DISCO DIAMANTE SEGMENTADO 7' USO GENERAL",
  "Disco de diamante, rin continuo, 4-1/2'",
  "Disco de diamante, rin segmentado, 4'",
  "Disco de diamante, rin turbo, 4'"]

 "Disco de diamante, rin continuo, 7'"


In [19]:
print(findProduct('Barreta uña 3/4 x 75cms BU75 Truper'))

[ "Barreta de punta de 7/8' x 125 cm",
  "Barreta de punta de 7/8' x 100 cm",
  "Barreta de punta de 1-1/4' x 175 cm",
  "Barreta de una de 3/4' x 75 cm",
  "Barreta de una de 3/4' x 45 cm",
  "Barreta de una de 3/4' x 90 cm",
  "Barreta de punta de 1' x 150 cm",
  "Barreta de una de 3/4' x 60 cm",
  "Barreta de punta de 1' x 175 cm, escoplo ancho",
  "Barreta de punta de 1' x 150 cm, escoplo ancho"]

 "Barreta de una de 3/4' x 75 cm"


In [31]:
lst = [
    "Barreta uña 3/4 x 75cms BU75 Truper",
    "Disco diamante 7x7/8 rin continuo uso gral ddt-804c truper",
    "Broca p/router pecho paloma 1-1/8\"x13mm BROUPP11/8X Truper",
]

print(findListOfProducts(lst))

[ "Barreta de una de 3/4' x 45 cm",
  "Barreta de una de 3/4' x 75 cm",
  "Dado de 6 puntas de 13/16' para bujias, cuadro 3/8'",
  "Barreta de punta de 7/8' x 125 cm",
  "Barreta de punta de 7/8' x 100 cm",
  "Barreta de punta de 1' x 175 cm, escoplo ancho",
  "Dado de 6 puntas largas de 13/16' para bujias, cuadro 1/2'",
  "Barreta de punta de 1' x 150 cm, escoplo ancho",
  "Broca para router, collarin, 7/8' con balero",
  "Broca para router, doble romana, 1-3/8' con balero",
  "Barreta de una de 3/4' x 60 cm",
  "Broca para router, pecho paloma, 1-3/8' con balero",
  "Broca para router, corte recto, 1/2' con balero",
  "Broca para router, pecho paloma, 1-1/8' con balero",
  "Barreta de una de 3/4' x 90 cm",
  "Barreta de punta de 1-1/4' x 175 cm",
  "Broca para router, bocel cuarto, 1-1/8' con balero",
  "Dado de 6 puntas, cuadro 1/2' de 12mm",
  "Dado de 6 puntas, cuadro 1/2' de 12mm",
  "Barreta de una de 1/2' x 30 cm",
  "Dado de 6 puntas de 5/8' para bujias, cuadro 3/8'",
  "Dado 

In [33]:
lst = [
    "Bolsa 50 remaches 3.2x6.4mm Truper",
    "Bolsa 50 remaches 3.2x11.10mm Truper",
    "Bolsa 50 remaches 4.8x21mm Truper",
    "Bolsa 50 remaches 6.4x15.9mm Truper",
    "Bolsa 50 remaches 6.4x19.1mm Truper",
    "Bolsa 50 remaches 4.8x21mm Truper",
    "Bolsa 50 remaches 4.8x14.5mm Truper"
]

print(findListOfProducts(lst))

[ 'Banda de lija 4x24 gr .36',
  'Banda de lija 4x24 gr .40',
  'Banda de lija 4x24 gr .60',
  "Manguera 4 capas  de 5/8'",
  'Banda de lija 4x24 gr .50',
  "Manguera 1', super reforzada, 4 capas, 50 m",
  "Manguera 3/4', super reforzada, 4 capas, 100 m",
  "BROCHA MGO/MADERA 1'   CERDA NAT 48X12MM",
  'CINCHOPLASTICO 500x4.6x1.5MM BOLSA C/25',
  'CINCHOPLASTICO 500x4.6x1.5MM BOLSA C/25',
  'CINCHOPLASTICO 300x4.6x1.5MM BOLSA C/50',
  'CINCHOPLASTICO 300x4.6x1.5MM BOLSA C/50',
  'Clavos p/clavadora neumatica CLNEU-2-1/2x, 40mm, 2500 pzas',
  'CINCHOPLASTICO 450x4.6x1.5MM BOLSA C/25',
  'CINCHOPLASTICO 450x4.6x1.5MM BOLSA C/25',
  'Clavos p/clavadora neumatica CLNEU-2-1/2x, 50mm, 2500 pzas',
  'Clavos p/clavadora neumatica CLNEU-2-1/2x, 30mm, 2500 pzas',
  'CINCHOPLASTICO 160x4.6x1.5MM BOLSA C/50',
  'CINCHOPLASTICO 160x4.6x1.5MM BOLSA C/50',
  'Remaches, bolsa con 50 piezas,6.4x19.1mm',
  "Manguera 5/8', super reforzada 4 capas, conex. de metal,25m",
  'Clavos p/clavadora neumatica CLN

In [40]:
findProduct("Bolsa 50 remaches 3.2x6.4mm Truper", 100)

[ 'CINCHOPLASTICO 300x4.6x1.5MM BOLSA C/50',
  'CINCHOPLASTICO 300x4.6x1.5MM BOLSA C/50',
  'CINCHOPLASTICO 500x4.6x1.5MM BOLSA C/25',
  'CINCHOPLASTICO 500x4.6x1.5MM BOLSA C/25',
  'CINCHOPLASTICO 450x4.6x1.5MM BOLSA C/25',
  'CINCHOPLASTICO 450x4.6x1.5MM BOLSA C/25',
  'CINCHOPLASTICO 368x4.6x1.5MM BOLSA C/25',
  'CINCHOPLASTICO 368x4.6x1.5MM BOLSA C/25',
  'CINCHOPLASTICO 160x4.6x1.5MM BOLSA C/50',
  'CINCHOPLASTICO 160x4.6x1.5MM BOLSA C/50',
  'CINCHOPLASTICO 200x3.5x1.5MM BOLSA C/50',
  'CINCHOPLASTICO 200x3.5x1.5MM BOLSA C/50',
  'CINCHOPLASTICO 200x3.5x1.5MM BOLSA C/50',
  'CINCHOPLASTICO 150x3.5x1.1MM BOLSA C/50',
  'CINCHOPLASTICO 150x3.5x1.1MM BOLSA C/50',
  'CINCHOPLASTICO 100x2.5x1.1MM BOLSA C/100',
  'CINCHOPLASTICO 100x2.5x1.1MM BOLSA C/100',
  'Clavos p/clavadora neumatica CLNEU-2-1/2x, 50mm, 2500 pzas',
  'Banda de lija 4x24 gr .36',
  'Clavos p/clavadora neumatica CLNEU-2-1/2x, 30mm, 2500 pzas',
  'Clavos p/clavadora neumatica CLNEU-2-1/2x, 40mm, 2500 pzas',
  'Piedra 

" 'Remaches, bolsa con 50 piezas,4.8x21mm'"

In [41]:
def findCategory(items):
    categories = """Palas nacionales
    Abrazaderas
    Conexiones de manguera para jardin
    Dados y accesorios 3/8"
    Dados y accesorios 1/2"
    Dados y accesorios 1/4"
    Cajas de herramientas
    Bombas para agua
    Bombas de Extraccion
    Carretillas nacionales y Latam
    Focos ahorradores
    Equipo de taller automotriz
    Portalamparas
    Lamparas de emergencia
    Focos de halogeno
    Repuestos para reflectores
    Multicontactos y adaptadores
    Multicontactos en barra y de figuras
    Extensiones electricas
    Luminarios Volteck Lait
    Lamparas de taller
    Cables pasa corriente
    Arcos para seguetas
    Fumigadores y atomizadores nacionales
    Focos para reflector
    Arcos jardineros nacionales
    Machetes
    Ganchos para perfocel
    Escaleras
    Clavijas y contactos industriales para cable
    Aspersores
    Martillos
    Sujetadores
    Accesorios de cerrajeria
    Pistolas de riego
    Azadones de ojo
    Cascos de seguridad
    Porta mangueras
    Mangueras para compresor
    Multimetros
    Accesorios para auto
    Cepillos para Calzado
    Mangos de repuesto
    Herramientas para niños
    Inversores de corriente
    Linternas
    Desarmadores
    Arneses y cables de proteccion
    Remaches en bolsa
    Adaptadores para discos
    Rodillos para pintar
    Azadones
    Azadones de zanco
    Marros nacionales
    Marros y tajaderas USA
    Refacciones de carretillas
    Maquinas estacionarias
    Ruedas para diablo
    Accesorios neumaticos
    Pintura en aerosol
    Cepillos de alambre
    Discos abrasivos
    Brocas (fresas) para router
    Fundas
    Flexometros
    Barretas, grifas y cuñas nacionales
    Puntas para desarmador
    Goggles y Lentes
    Fajas y correas para carga
    Lonas
    Seguetas para sierra sable
    Barretas y grifas nacionales
    Verniers y calibradores
    Fumigadores importados
    Guantes
    Mangueras de jardin armadas
    Maquinas portatiles
    Equipos de corte y soldadura
    Gavetas de plastico
    Herramientas agricolas
    Hachas  Nacionales
    Limas
    Espumas expansivas
    Herramientas de jardin nacional
    Cintas para ducto
    Esmeriles
    Seguridad vial
    Hidrolavadoras accesorios
    Llaves y juegos
    Berbiquies
    Desbrozadoras
    Llaves para plomero
    Bieldos nacionales
    Carbones para maquinas
    Candados
    Brocas para madera
    Pistolas para pintar
    Aspiradoras
    Ganchos metalicos
    Escobas para jardin nacionales
    Plastiprotectores
    Brocas para metal
    Mangueras de jardin en rollo
    Cavadores Nacionales y Latam
    Cortadores de varilla
    Herramientas neumaticas
    Brocas para concreto
    Embudos plasticos
    Niveles
    Cucharas de albañil
    Brocas SDS
    Juegos mixtos de brocas
    Brocas planas
    Podadoras (Refacciones y accesorios)
    Llanas y flotas
    Machuelos y tarrajas
    Porta herramientas
    Cardas
    Discos de diamante
    Lijas
    Piedras de asentar
    Discos laminados
    Puntas montadas
    Brocas para vidrio
    Prensas y sargentos
    Cintas de montaje
    Engrapadoras y clavadoras neumaticas
    Cortadores de azulejo
    Maquinas de soldar
    Pistola para pintar (Refacciones y accesorios)
    Plomadas
    Tenazas
    Espatulas
    Brocasierras bimetalicas
    Cinceles SDS
    Tanques de gas y boquillas
    Compresores ( refacciones )
    Separadores de loseta
    Repuesto cortador de azulejo
    Revolvedores de pintura
    Hidrolavadoras
    Bombas de Aire
    Cepillos carpintero
    Cepillo (Refacciones)
    Cinceles y punzones
    Accesorios para soldar
    Escuadras magneticas
    Brocas escalonadas
    Discos de sierra
    Cintas de aluminio
    Cinceles
    Punzones
    Cortasetos
    Copas de diamante
    Destapacaños
    Soldaduras
    Cinchos de plastico y de contacto
    Baterias y cargadores de maquinas
    Pinzas
    Raspadores con navaja
    Proteccion auditiva
    Piedras para esmeril
    Dados y accesorios 3/4"
    Brochas y cepillos para pintor
    Accesorios para desbrozadora
    Cintas de aislar
    Cintas de teflon y pasta sella roscas
    Cintas antiderrapantes
    Cintas de empaque
    Masking tape
    Bandas, cintas delimitadoras, malla y banderola
    Adhesivos
    Cintas largas
    Tijeras a dos manos y cortadores de ramas altas
    Cautines
    Cuchillos
    Grilletes y tensores
    Cables de acero
    Hilo para albañil
    Lubricantes, aceites y grasas
    Cortapernos y Suajadora
    Cortatubo y avellanadores
    Cortapernos (Repuestos)
    Amarradores de varillas
    Dobladores de tubo
    Cargadores de baterias
    Juegos para mecanico Truper
    Polipastos
    Selladores
    Rodilleras
    Cortadores de vidrio
    Tirolera
    Ruedas flap
    Sopladoras
    Accesorios maquinas
    Caretas, gafas y protectores de carnaza
    Chiflones
    Llaves de cadena, banda y universal para filtros de aceite
    Llaves de jardin
    Dados y accesorios
    Matracas y accesorios
    Grapas y clavos para engrapadoras y clavadoras neumaticas
    Llaveros
    Chalecos y sombreros reflejantes
    Basculas
    Llaves Allen y Torx
    Proteccion respiratoria
    Microalambres
    Copas de desbaste
    Compresores
    Navajas
    Juegos de herramientas Truper
    Overoles
    Probadores de corriente
    Afiladores
    Diablos de carga
    Herramientas automotrices
    Protectores faciales
    Pinzas portalectrodo y para tierra
    Estantes metalicos
    Generadores
    Electrodos
    Escuadras
    Calibradores
    Medidores de aire
    Impermeables y ponchos
    Extractores
    Llaves de banda y universal para filtro de aceite
    Formones
    Fumigadores
    Tijeras para poda a una mano nacional
    Malacates
    Accesorios para cable
    Bandolas
    Gatos de patin
    Gatos de tijeras
    Gatos de botella
    Cadenas
    Graseras y Aceiteras
    Guadañas
    Guia de Inglete
    Parrillas a gas
    Focos para linterna
    GARDEN TOOLS
    Herramientas Classic Gardener
    Molinos
    Mangueras de PVC para gas
    Mangueras flexibles para agua y gas
    Hoces
    Rayador, volteador y jalador
    Flotas
    Anzuelos para pescar
    Zapatos industriales
    Broqueros y llaves
    Llaves de cruz y de artilleria
    Llaves ajustables
    Juegos de herramientas Pretul
    Multiherramientas
    Odometros
    LLaves Stilson
    Hilos para desbrozadora
    Taquetes
    Seguetas para caladora
    Accesorios mototool
    Motosierras"""

    itemsString = ""
    for i in items:
        itemsString += i        

    prompt = f"""Human: You are an assistant at a hardware store tasked with stocking newly arrived items. Each item must be stored in their
    corresponding category. The list of categories is enclosed in <categories> tags. The list of items that you must categorize is enclosed in
    <items> tags. Respond only with a list of each item and their category.

    <categories>
    {categories}
    </categories>

    <items>
    {itemsString}
    </items>

    Assistant:
    """

    body = json.dumps({"prompt": prompt, "max_tokens_to_sample": 1000})
    modelId = "anthropic.claude-instant-v1"
    accept = "application/json"
    contentType = "application/json"

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    responsebody = json.loads(response.get("body").read())
    return responsebody.get("completion")

In [42]:
items = [
    "Dado de 6 puntas, cuadro 3/4' de 50mm",
    "CADENA TIPO VICTOR NO.2 DE 2.3MM X 30M",
    "Fumigador de mochila, 12 lts",
    "Brocha 3' con mango de plástico",
    "Dado de 12 puntas, cuadro 3/4' de 1-5/8'",
    "Lampara recargable de led alta potencia, 650 lm, 5 W",
    "PUNZON BOTADOR RECTO 3/8'X 3/16'",
    "Broca para router, corte recto, 1/2' con balero",
]

print(findCategory(items))

 Dado de 6 puntas, cuadro 3/4' de 50mm - Dados y accesorios 3/4"
CADENA TIPO VICTOR NO.2 DE 2.3MM X 30M - Cadenas
Fumigador de mochila, 12 lts - Fumigadores y atomizadores nacionales
Brocha 3' con mango de plástico - Brochas y cepillos para pintor
Dado de 12 puntas, cuadro 3/4' de 1-5/8' - Dados y accesorios 3/4" 
Lampara recargable de led alta potencia, 650 lm, 5 W - Linternas
PUNZON BOTADOR RECTO 3/8'X 3/16' - Punzones
Broca para router, corte recto, 1/2' con balero - Brocas para madera


In [47]:
query = "Fumigadores y atomizadores nacionales"

response = retrieve(query, kb_id, 100)
items = get_contexts(response['retrievalResults'])
pp.pprint(items)

[ 'Traba de la palanca para fumigadores FUM-12, 16 y 20',
  'Boquilla de salida para fumigadores FUM-12, 16 y 20',
  'Lanza para fumigadores FUM-15, 16 y 20',
  'Boquilla JD-12, completa para fumigadores FUM-12, 16 y 20',
  'Manija para fumigadores FUM-15, 16 y 20',
  'Tapa de la boquilla para fumigadores FUM-12, 16 y 20',
  'Tuerca union para fumigadora FM-425',
  'Camara con embolo para fumigadores FUM-12, 16 y 20',
  'Hebilla para fumigadores FUM-15, 16 y 20',
  'Cilindro de la camara para fumigadores FUM-12, 16 y 20',
  'Lanza LP-3 para fumigadores FUM-12,16 y 20',
  'Embolo (Santopreno) para fumigadores FUM-12,16 y 20',
  'Boquilla JD-12 para fumigadores FUM-12, 16 y 20',
  'Aguja LP3 para fumigadores FUM-12, 16 y 20',
  'Resorte para fumigadores FUM-15, 16 y 20',
  'Tuerca conica para fumigadores FUM-12, 16 y 20',
  'Union con junta conica para fumigadores FUM-12, 16 y 20',
  'Abrazadera SX9-18 para fumigadora FM-425',
  'Tapa para fumigadores FUM-15, 16 y 20',
  'Varilla naranja

In [48]:
query = "Remaches en bolsa"

response = retrieve(query, kb_id, 100)
items = get_contexts(response['retrievalResults'])
pp.pprint(items)

[ 'Remaches, bolsa con 50 piezas, 4x7.10mm',
  'Remaches, bolsa con 50 piezas, 3.2x8.0mm',
  'Remaches, bolsa con 50 piezas, 3.2x9.6mm',
  'Remaches, bolsa con 50 piezas, 4x16.50mm',
  'Remaches, bolsa con 50 piezas,4.8x24.1mm',
  'Remaches, bolsa con 50 piezas, 3.2x6.4mm',
  'Remaches, bolsa con 50 piezas, 4x10.20mm',
  'Remaches, bolsa con 50 piezas, 4x13.4mm',
  'Remaches, bolsa con 50 piezas,4.8x8.0mm',
  'Remaches, bolsa con 50 piezas,6.4x19.1mm',
  'Remaches, bolsa con 50 piezas,4.8x11.0mm',
  'Remaches, bolsa con 50 piezas,4.8x21mm',
  'Remaches, bolsa con 50 piezas,6.4x15.9mm',
  'Remaches, bolsa con 50 piezas,6.4x12.7mm',
  'Remaches, bolsa con 50 piezas,3.2x12.7mm',
  'Remaches, bolsa con 50 piezas,4.8x17.7mm',
  'Remaches, bolsa con 50 piezas,4.8x14.5mm',
  'Remaches, bolsa con 50 piezas,4.8x14.5mm',
  'Remaches, bolsa con 50 piezas,4.8x14.5mm',
  'Remaches,bolsa con 50 piezas,3.2x16.1mm',
  'Remaches,bolsa con 50 piezas,3.2x11.10mm',
  'Repuesto de empaques/boquillas',
  'B

In [52]:
print(findProduct("Bolsa 50 remaches 3.2x6.4mm Truper", 100))

[ 'CINCHOPLASTICO 300x4.6x1.5MM BOLSA C/50',
  'CINCHOPLASTICO 300x4.6x1.5MM BOLSA C/50',
  'CINCHOPLASTICO 500x4.6x1.5MM BOLSA C/25',
  'CINCHOPLASTICO 500x4.6x1.5MM BOLSA C/25',
  'CINCHOPLASTICO 450x4.6x1.5MM BOLSA C/25',
  'CINCHOPLASTICO 450x4.6x1.5MM BOLSA C/25',
  'CINCHOPLASTICO 368x4.6x1.5MM BOLSA C/25',
  'CINCHOPLASTICO 368x4.6x1.5MM BOLSA C/25',
  'CINCHOPLASTICO 160x4.6x1.5MM BOLSA C/50',
  'CINCHOPLASTICO 160x4.6x1.5MM BOLSA C/50',
  'CINCHOPLASTICO 200x3.5x1.5MM BOLSA C/50',
  'CINCHOPLASTICO 200x3.5x1.5MM BOLSA C/50',
  'CINCHOPLASTICO 200x3.5x1.5MM BOLSA C/50',
  'CINCHOPLASTICO 150x3.5x1.1MM BOLSA C/50',
  'CINCHOPLASTICO 150x3.5x1.1MM BOLSA C/50',
  'CINCHOPLASTICO 100x2.5x1.1MM BOLSA C/100',
  'CINCHOPLASTICO 100x2.5x1.1MM BOLSA C/100',
  'Clavos p/clavadora neumatica CLNEU-2-1/2x, 50mm, 2500 pzas',
  'Banda de lija 4x24 gr .36',
  'Clavos p/clavadora neumatica CLNEU-2-1/2x, 30mm, 2500 pzas',
  'Clavos p/clavadora neumatica CLNEU-2-1/2x, 40mm, 2500 pzas',
  'Piedra 

In [53]:
print(findProduct("Pala redonda c/mgo y pryp4truper"))

[ 'Jgo. De 9 llaves allen mm extralargas',
  'Pala azada, redonda, # 5',
  'Pala azada, redonda, # 6',
  "Perico profesional, pavonado 4', Truper Expert en tarjeta",
  'Guantes uso gral. Carnaza y loneta, doble refuerzo, chicos',
  "Perico profesional, pavonado 10', Truper Expert en tarjeta",
  'Guantes de nylon recubiertos con poliuretano, chicos',
  "Perico profesional, pavonado 24', Truper Expert en tarjeta",
  'Repuesto para podadora POMA-15, cuna',
  "MANGUERA 4 CAPAS  DE 5/8' REFORZADA"]

 Pala azada, redonda, # 5
Pala azada, redonda, # 6

The customer seemed to be looking for a round shovel or spade. Based on the size mentioned (#5 or #6), I've listed two similar items from the store inventory that match the customer's description.
